# Week 12 – Agents, Tools, Memory, Chains (LangChain + LangGraph)

이 노트북은 다음 세 가지 축으로 구성된 예제 코드를 포함합니다.

1. **LangChain**: 기본 Chat, Chain, Memory, Tool, Agent (`create_agent`)
2. **LangGraph**: StateGraph 기반 간단한 Agent 그래프

> ⚠️ 주의: 실제 실행을 위해서는 `OPENAI_API_KEY`, `Langsmith` 관련 환경변수 설정이 필요합니다.


## 1. 환경 설정 & 패키지 설치

실습 환경에 맞게 한 번만 설치하면 됩니다. (Colab, 로컬 venv 등)


In [12]:
pip install langsmith langchain langchain_openai langgraph langchain-core

Note: you may need to restart the kernel to use updated packages.


## 2. 공통 설정: 환경변수 & 모델 헬퍼

- `.env` 파일에서 OpenAI / LangFuse 키를 로딩
- 공통 Chat 모델 생성 헬퍼 함수 정의


In [4]:
# 설치 (환경마다 한 번만)
# %pip install -q langsmith langchain_core langchain_openai langgraph

import os
from dotenv import load_dotenv

load_dotenv()  # .env 로딩

# LangSmith 기본 체크
print("LANGSMITH_TRACING:", os.getenv("LANGSMITH_TRACING"))
print("LANGSMITH_PROJECT:", os.getenv("LANGSMITH_PROJECT"))
print("LANGSMITH_API_KEY set?:", bool(os.getenv("LANGSMITH_API_KEY")))

# OpenAI 키 체크
print("OPENAI_API_KEY set?:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ WARNING: OPENAI_API_KEY가 설정되지 않았습니다. 실제 실행 전 반드시 설정하세요.")

# LangChain / LangGraph 공통에서 사용할 기본 모델 헬퍼
from langchain_openai import ChatOpenAI

def get_chat_model(
    model_name: str = "gpt-4o-mini",
    temperature: float = 0.1,
):
    '''
    공통 Chat 모델 생성 헬퍼.
    - 필요에 따라 로컬 LLM (ex. ollama) 으로 교체 가능.
    '''
    return ChatOpenAI(
        model=model_name,
        temperature=temperature,
    )


LANGSMITH_TRACING: true
LANGSMITH_PROJECT: ajou-ai-practical-project10-llmops
LANGSMITH_API_KEY set?: True
OPENAI_API_KEY set?: True


## 3. LangChain 기본 Chat & Chain

가장 기본적인 **프롬프트 → LLM** 체인을 정의하고 실행해 봅니다.
- `ChatPromptTemplate` + `ChatOpenAI` + `|` 연산자로 구성


In [5]:
from langchain_core.prompts import ChatPromptTemplate

# 3-1. 간단한 프롬프트 템플릿
basic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant for LLMOps students."),
        ("human", "Explain the concept of {topic} in 3 sentences."),
    ]
)

# 3-2. 체인 구성 (Prompt -> Model)
basic_chain = basic_prompt | get_chat_model()

# 3-3. 테스트 실행
if OPENAI_API_KEY:
    result = basic_chain.invoke({"topic": "LangChain agents"})
    print(result.content)
else:
    print("OPENAI_API_KEY 미설정: 체인 예시는 구조만 확인하세요.")


LangChain agents are components designed to enable language models to interact with external tools and data sources, allowing them to perform complex tasks beyond simple text generation. They utilize a combination of a language model, a decision-making mechanism, and various tools or APIs to execute actions based on user input or specific prompts. This architecture empowers developers to create applications that can reason, retrieve information, and automate workflows by leveraging the capabilities of language models in a structured manner.


## 4. LangChain Memory – `RunnableWithMessageHistory`

대화형 에이전트에서 **대화 Memory** 를 관리하기 위해 `RunnableWithMessageHistory` 를 사용합니다.

- 세션 ID 별로 메시지 히스토리를 저장
- LangChain의 `messages` 입력/출력을 자동으로 관리


In [6]:
from typing import Dict, List
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

# 세션별 메모리를 관리하는 간단한 in-memory store
_chat_store: Dict[str, InMemoryChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in _chat_store:
        _chat_store[session_id] = InMemoryChatMessageHistory()
    return _chat_store[session_id]

# 메모리 포함 Chat 체인 정의
memory_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that remembers the conversation."),
        ("human", "{input}"),
    ]
)

memory_chain = memory_prompt | get_chat_model()

conversation = RunnableWithMessageHistory(
    memory_chain,
    get_session_history,
    input_messages_key="input",   # 입력에서 대화 텍스트가 있는 key
    history_messages_key="history",  # 히스토리 저장 key
)

if OPENAI_API_KEY:
    print("=== Session A: 첫 질문 ===")
    res1 = conversation.invoke(
        {"input": "Hi, I am Sungjae. Please remember my name."},
        config={"configurable": {"session_id": "session-a"}},
    )
    print(res1.content)

    print("\n=== Session A: 후속 질문 ===")
    res2 = conversation.invoke(
        {"input": "What is my name?"},
        config={"configurable": {"session_id": "session-a"}},
    )
    print(res2.content)
else:
    print("OPENAI_API_KEY 미설정: Memory 체인 구조만 확인하세요.")


=== Session A: 첫 질문 ===
Hi, Sungjae! I’ll remember your name. How can I assist you today?

=== Session A: 후속 질문 ===
I'm sorry, but I don't have access to your name or any personal information. How can I assist you today?


## 5. LangChain Tools – `@tool` & `ToolRuntime`

에이전트가 호출할 수 있는 **Tool** 을 정의합니다.

- 간단한 `calculator` / `get_server_time`
- `ToolRuntime` 을 활용해 현재 대화 상태에 접근하는 `summarize_conversation`


In [7]:
from datetime import datetime
from langchain.tools import tool, ToolRuntime

@tool
def calculator(expression: str) -> str:
    '''주어진 수식(expression)을 Python eval 로 계산합니다 (간단 예제).'''
    try:
        # ⚠️ 실제 서비스에서는 eval 대신 안전한 파서 사용 필요
        result = eval(expression, {"__builtins__": {}})
        return f"{expression} = {result}"
    except Exception as e:
        return f"Failed to evaluate expression: {e}"

@tool
def get_server_time() -> str:
    '''서버 현재 시간을 ISO 포맷으로 반환합니다.'''
    return datetime.now().isoformat()

@tool
def echo_upper(text: str) -> str:
    '''텍스트를 대문자로 변환하여 반환합니다.'''
    return text.upper()

# ToolRuntime 활용 예시
@tool
def summarize_conversation(runtime: ToolRuntime) -> str:
    '''지금까지의 대화를 짧게 요약합니다.'''
    messages = runtime.state.get("messages", [])
    # messages 는 BaseMessage 리스트
    user_turns = [m.content for m in messages if m.type == "human"]
    ai_turns = [m.content for m in messages if m.type == "ai"]
    return (
        f"User said {len(user_turns)} things, "
        f"Assistant replied {len(ai_turns)} times. "
        "대화 내용은 에이전트가 직접 요약해서 사용자에게 전달하세요."
    )

tools = [calculator, get_server_time, echo_upper, summarize_conversation]
tools


[StructuredTool(name='calculator', description='주어진 수식(expression)을 Python eval 로 계산합니다 (간단 예제).', args_schema=<class 'langchain_core.utils.pydantic.calculator'>, func=<function calculator at 0x107ae7740>),
 StructuredTool(name='get_server_time', description='서버 현재 시간을 ISO 포맷으로 반환합니다.', args_schema=<class 'langchain_core.utils.pydantic.get_server_time'>, func=<function get_server_time at 0x1337d3f60>),
 StructuredTool(name='echo_upper', description='텍스트를 대문자로 변환하여 반환합니다.', args_schema=<class 'langchain_core.utils.pydantic.echo_upper'>, func=<function echo_upper at 0x1337d2520>),
 StructuredTool(name='summarize_conversation', description='지금까지의 대화를 짧게 요약합니다.', args_schema=<class 'langchain_core.utils.pydantic.summarize_conversation'>, func=<function summarize_conversation at 0x1337f4cc0>)]

## 6. LangChain Agent – `create_agent`

새로운 `create_agent` API 를 활용해 **ReAct 패턴 기반 에이전트**를 구성합니다.

- LLM + Tools 조합
- `messages` 기반 입력
- `.invoke()` / `.stream()` 으로 실행


In [8]:
from langchain.agents import create_agent

agent_model = get_chat_model(model_name="gpt-4o-mini", temperature=0.2)

system_prompt = (
    "You are an LLMOps teaching assistant.\n"
    "You can use tools to calculate expressions, get server time, "
    "and transform text to UPPERCASE.\n"
    "When appropriate, think step by step and show intermediate reasoning briefly."
)

agent = create_agent(
    model=agent_model,
    tools=tools,
    system_prompt=system_prompt,
)

def run_simple_agent(query: str):
    if not OPENAI_API_KEY:
        print("OPENAI_API_KEY 미설정: 에이전트 구조만 확인하세요.")
        return

    events = agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
    )
    final = None
    for event in events:
        final = event
    if final:
        # 마지막 메시지 출력
        print(final["messages"][-1].content)

# 간단 테스트
run_simple_agent("지금 시간이 몇 시인지 알려주고, 2 * (3 + 5)의 결과도 계산해줘.")


현재 시간은 2025년 11월 17일 22시 39분입니다.  
또한, \( 2 \times (3 + 5) \)의 결과는 16입니다.


## 7. LangChain Agent + Memory 결합

앞에서 정의한 `agent` 를 다시 감싸서 **세션 기반 Memory** 를 추가합니다.


In [9]:
from langchain_core.runnables import Runnable

# agent 는 이미 LangGraph 기반 Runnable (messages -> messages)
# 이를 RunnableWithMessageHistory 로 감싸서 대화 히스토리 유지
agent_with_memory = RunnableWithMessageHistory(
    agent,
    get_session_history,
    input_messages_key="messages",  # create_agent 는 messages 기반 입력
    history_messages_key="history",
)

def chat_with_agent(session_id: str, user_message: str):
    messages = [{"role": "user", "content": user_message}]
    result = agent_with_memory.invoke(
        {"messages": messages},
        config={"configurable": {"session_id": session_id}},
    )
    return result["messages"][-1].content

if OPENAI_API_KEY:
    print("=== Session AGENT-1: 첫 질문 ===")
    print(chat_with_agent("agent-session-1", "내 이름을 성재라고 기억해줘."))

    print("\n=== Session AGENT-1: 후속 질문 ===")
    print(chat_with_agent("agent-session-1", "내 이름이 뭐라고 했지?"))
else:
    print("OPENAI_API_KEY 미설정: Agent + Memory 예시는 구조만 확인하세요.")


=== Session AGENT-1: 첫 질문 ===
알겠습니다, 성재님! 당신의 이름을 기억하겠습니다.

=== Session AGENT-1: 후속 질문 ===
아직 당신의 이름을 말씀해 주신 적이 없습니다. 이름을 알려주시면 기억하겠습니다!


## 8. LangGraph – 기본 StateGraph 예제

LangGraph 를 사용해 **명시적인 StateGraph** 를 구성합니다.

- `State` (TypedDict) 정의
- `messages` 필드를 `add_messages` 리듀서로 관리
- `run_llm` 노드를 거치는 단순한 그래프


In [10]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# 노드 함수: 마지막 메시지를 보고 모델 호출
def run_llm_node(state: ChatState) -> ChatState:
    model = get_chat_model()
    response = model.invoke(state["messages"])
    return {"messages": [response]}

# 그래프 구성
graph_builder = StateGraph(ChatState)
graph_builder.add_node("llm", run_llm_node)
graph_builder.add_edge("llm", END)
graph_builder.set_entry_point("llm")

chat_graph = graph_builder.compile()

# 실행 예시
if OPENAI_API_KEY:
    from langchain_core.messages import HumanMessage

    events = chat_graph.stream(
        {"messages": [HumanMessage(content="Explain what LangGraph is in 2 sentences.")]}, 
        stream_mode="values",
    )
    final_state = None
    for s in events:
        final_state = s
    print(final_state["messages"][-1].content)
else:
    print("OPENAI_API_KEY 미설정: LangGraph 예시는 구조만 확인하세요.")


LangGraph is a framework designed to facilitate the development and deployment of language models by providing a structured way to represent and manipulate language data. It enables users to create complex language processing workflows by integrating various components and tools, enhancing the efficiency and effectiveness of natural language understanding tasks.


## 9. LangGraph + Tools – Tool 호출 노드

LangGraph 내에서 **Tool 을 직접 호출하는 노드**를 만들어,
- 사용자의 요청을 분석해서 Tool 실행
- 결과를 메시지에 추가

하는 패턴을 보여줍니다.


In [11]:
from langchain_core.tools import Tool

# 기존 LangChain @tool 로 만든 것들을 LangGraph 노드에서 직접 사용할 수 있음
simple_tool_map = {t.name: t for t in tools}

def tool_router_node(state: ChatState) -> ChatState:
    '''
    아주 단순한 규칙 기반 라우터 예시:
    - "CALC:" 로 시작하면 calculator 사용
    - "TIME" 을 포함하면 get_server_time 사용
    - 그 외에는 echo_upper 사용
    '''
    from langchain_core.messages import HumanMessage, AIMessage

    last_message = state["messages"][-1]
    if not isinstance(last_message, HumanMessage):
        return {"messages": []}

    text = last_message.content.strip()
    if text.upper().startswith("CALC:"):
        expr = text.split("CALC:", 1)[1].strip()
        tool_res = simple_tool_map["calculator"].invoke({"expression": expr})
        reply = f"[CALC RESULT] {tool_res}"
    elif "TIME" in text.upper():
        tool_res = simple_tool_map["get_server_time"].invoke({})
        reply = f"[TIME RESULT] {tool_res}"
    else:
        tool_res = simple_tool_map["echo_upper"].invoke({"text": text})
        reply = f"[ECHO RESULT] {tool_res}"

    return {"messages": [AIMessage(content=reply)]}

# 새로운 그래프: tool_router_node 사용
graph_builder2 = StateGraph(ChatState)
graph_builder2.add_node("tool_router", tool_router_node)
graph_builder2.add_edge("tool_router", END)
graph_builder2.set_entry_point("tool_router")

tool_graph = graph_builder2.compile()

# 실행 예시
if OPENAI_API_KEY:
    from langchain_core.messages import HumanMessage

    print("=== CALC 예시 ===")
    for s in tool_graph.stream(
        {"messages": [HumanMessage(content="CALC: 2 * (3 + 5)")]},
        stream_mode="values",
    ):
        pass
    print(s["messages"][-1].content)

    print("\n=== TIME 예시 ===")
    for s in tool_graph.stream(
        {"messages": [HumanMessage(content="What TIME is it?")]},
        stream_mode="values",
    ):
        pass
    print(s["messages"][-1].content)
else:
    print("OPENAI_API_KEY 미설정: Tool Router 그래프 구조만 확인하세요.")


=== CALC 예시 ===
[CALC RESULT] 2 * (3 + 5) = 16

=== TIME 예시 ===
[TIME RESULT] 2025-11-17T22:39:13.112186


## 13. 정리

이 노트북에서 다룬 내용 요약:

1. **LangChain**
   - 기본 Chat Chain (`ChatPromptTemplate` + `ChatOpenAI`)
   - `RunnableWithMessageHistory` 기반 Memory
   - `@tool`, `ToolRuntime` 기반 Tool 정의
   - `create_agent` 로 Agent 구축 + Memory 결합

2. **LangGraph**
   - `StateGraph`, `add_messages` 기반 ChatState 정의
   - LLM 노드 / Tool Router 노드 예제
